### Connect to your project

NOTE: When running the first cell you will be prompted to pick your Python environment, for the pre-built container choose:
 * **Python Environment** on the first dropdown
 * **Python 3.10.x** on the second dropdown

In [1]:
from azure.ai.generative import AIClient
from azure.identity import DefaultAzureCredential

# connects to project defined in the config.json file at the root of the repo
# use "ai init" to update this to point at your project
client = AIClient.from_config(DefaultAzureCredential())

Class AIClient: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Retrieve Azure OpenAI and Cognitive Services Connections and Set in Environment   

In [3]:
# Log into the Azure CLI (run az login --use-device code) before running this step!
default_aoai_connection = client.get_default_aoai_connection()
default_aoai_connection.set_current_environment()

# change this if you use different connection name
default_acs_connection = client.connections.get("Default_CognitiveSearch")
default_acs_connection.set_current_environment()

# change these if you use different deployment names
# if you do that, also update the deployment name in qna_simple/langchain_model.py 
chat_model_deployment = "gpt-35-turbo-16k-0613"
embedding_model_deployment = "text-ada-embedding-002-2"


Class WorkspaceConnection: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Build MLIndex Locally

In [4]:
from azure.ai.generative.operations._index_data_source import LocalSource, ACSOutputConfig
from azure.ai.generative.functions.build_mlindex import build_mlindex

# build the index using the product catalog docs from data/3-product-info
index = build_mlindex(
    output_index_name="product-info-cog-search-index",
    vector_store="azure_cognitive_search",
    embeddings_model = f"azure_open_ai://deployment/{embedding_model_deployment}/model/text-embedding-ada-002",
    data_source_url="https://product_info.com",
    index_input_config=LocalSource(input_data="../../data/3-product-info"),
    acs_config=ACSOutputConfig(
        acs_index_name="product-info-index-test1",
    ),
)

# register the index so that it shows up in the project
client.mlindexes.create_or_update(index)

### Implement co-pilot logic using Langchain and the MLIndex

In [10]:
from azureml.rag.mlindex import MLIndex

# convert MLIndex to a langchain retriever
index_langchain_retriever = MLIndex(
    client.mlindexes.get(name="product-info-cog-search-index", label="latest").path,
).as_langchain_retriever()

In [12]:
# create a QnA function that retrieves data and uses it as context to the LLM
def qna(question, temperature=0.0, number_of_docs=5, prompt_template=None):
    from langchain import PromptTemplate
    from langchain.chains import RetrievalQA
    from langchain.chat_models import AzureChatOpenAI

    llm = AzureChatOpenAI(
        deployment_name=chat_model_deployment,
        model_name="gpt-35-16k-turbo",
        temperature=temperature,
    )

    template = """
    System:
    You are an AI assistant helping users with queries related to outdoor outdooor/camping gear and clothing.
    Use the following pieces of context to answer the questions about outdoor/camping gear and clothing as completely, correctly, and concisely as possible.
    If the question is not related to outdoor/camping gear and clothing, just say Sorry, I only can answer question related to outdoor/camping gear and clothing. So how can I help? Don't try to make up an answer.
    If the question is related to outdoor/camping gear and clothing but vague ask for clarifying questions.
    Do not add documentation reference in the response.

    {context}

    ---

    Question: {question}

    Answer:"
    """
    prompt_template = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=index_langchain_retriever,
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": prompt_template,
        }
    )

    response = qa(question)

    return {
        "question": response["query"],
        "answer": response["result"],
        "context": "\n\n".join([doc.page_content for doc in response["source_documents"]])
    }
    

In [ ]:
result = qna("Which tent has the highest rainfly waterproof rating?")
print(result["answer"])

### Evaluate LLM Response on Larger Dataset

In [13]:
import json
import os

# Loading data
def load_jsonl(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f.readlines()]

path = os.path.join(os.getcwd() + "/data.jsonl")
data = load_jsonl(path)

In [17]:
from azure.ai.generative.evaluate import evaluate
from pprint import pprint

# Evaluate the default vs the improved system prompt to see if the improved prompt
# performs consistently better across a larger set of inputs
result = evaluate(
    evaluation_name="baseline-evaluation",
    asset=qna,# model_uri:
    data=data,
    task_type="qa",
    prediction_data="answer",
    truth_data="truth", # Optional
    metrics_config={
        "openai_params": {
            "api_version": "2023-05-15",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_type": "azure",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "deployment_id": chat_model_deployment
        },
        "questions": "question",
        "contexts": "context",
    },
    tracking_uri=client.tracking_uri,
)
pprint(result)

# Print a link to open the evalautions page in AI Studio
print(f"Open in AI Studio: https://ml.azure.com/projectEvaluation?flight=AiStudio,DeployChatWebapp,SkipAADRegistration/projectEvaluation&wsid=/subscriptions/{client.subscription_id}/resourceGroups/{client.resource_group_name}/providers/Microsoft.MachineLearningServices/workspaces/{client.project_name}")

test data is not a file but loaded data
Error logging data as dataset, continuing without it


Scoring failed for QA metric exact_match
Class: MissingDependencies
Message: evaluate package is not available. Please run pip install azureml-metrics[evaluate]
Scoring failed for QA metric bertscore
Class: MissingDependencies
Message: bert-score packages are not available. Please run pip install azureml-metrics[bert-score]
Failed to aggregate the scores for metric : bertscore with the following exception : ufunc 'add' did not contain a loop with signature matching types (dtype('<U19'), dtype('<U19')) -> None


{'artifacts': {'gpt_coherence': ['5',
                                 '5',
                                 '5',
                                 '5',
                                 '1',
                                 '5',
                                 '3',
                                 '5',
                                 '5',
                                 '3',
                                 '5',
                                 '5',
                                 '1'],
               'gpt_fluency': ['5',
                               '5',
                               '5',
                               '5',
                               '1',
                               '4',
                               '3',
                               '5',
                               '5',
                               '3',
                               '5',
                               '5',
                               '1'],
               'gpt_groundedness': [

### Evaluate copilot performance across different parameters

In [15]:
# Sweep over different values of temperature and number_of_docs to find the best value
# Evaluation results will be logged to project by setting tracking_uri
result = evaluate( 
    evaluation_name="qna-params-eval",
    asset=qna,
    data=data,
    task_type="qa",
    prediction_data="answer",
    truth_data="truth", # Optional
    metrics_config={
        "openai_params": {
            "api_version": "2023-05-15",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_type": "azure",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "deployment_id": chat_model_deployment
        },
        "questions": "question",
        "contexts": "context",
    },
    tracking_uri=client.tracking_uri,
    params={
        "temperature": [0.0, 0,1],
        "number_of_docs": [5, 10]
    }
)

pprint(result)

# Print a link to open the evalautions page in AI Studio
print(f"Open in AI Studio: https://ml.azure.com/projectEvaluation?flight=AiStudio,DeployChatWebapp,SkipAADRegistration/projectEvaluation&wsid=/subscriptions/{client.subscription_id}/resourceGroups/{client.resource_group_name}/providers/Microsoft.MachineLearningServices/workspaces/{client.project_name}")

test data is not a file but loaded data
Error logging data as dataset, continuing without it


Scoring failed for QA metric bertscore
Class: MissingDependencies
Message: bert-score packages are not available. Please run pip install azureml-metrics[bert-score]
Scoring failed for QA metric exact_match
Class: MissingDependencies
Message: evaluate package is not available. Please run pip install azureml-metrics[evaluate]
Failed to aggregate the scores for metric : bertscore with the following exception : ufunc 'add' did not contain a loop with signature matching types (dtype('<U19'), dtype('<U19')) -> None


test data is not a file but loaded data
Error logging data as dataset, continuing without it


Scoring failed for QA metric bertscore
Class: MissingDependencies
Message: bert-score packages are not available. Please run pip install azureml-metrics[bert-score]
Scoring failed for QA metric exact_match
Class: MissingDependencies
Message: evaluate package is not available. Please run pip install azureml-metrics[evaluate]
Failed to aggregate the scores for metric : bertscore with the following exception : ufunc 'add' did not contain a loop with signature matching types (dtype('<U19'), dtype('<U19')) -> None


test data is not a file but loaded data
Error logging data as dataset, continuing without it


...

test data is not a file but loaded data
Error logging data as dataset, continuing without it


Scoring failed for QA metric bertscore
Class: MissingDependencies
Message: bert-score packages are not available. Please run pip install azureml-metrics[bert-score]
Scoring failed for QA metric exact_match
Class: MissingDependencies
Message: evaluate package is not available. Please run pip install azureml-metrics[evaluate]
Failed to aggregate the scores for metric : bertscore with the following exception : ufunc 'add' did not contain a loop with signature matching types (dtype('<U19'), dtype('<U19')) -> None


test data is not a file but loaded data
Error logging data as dataset, continuing without it


Scoring failed for QA metric bertscore
Class: MissingDependencies
Message: bert-score packages are not available. Please run pip install azureml-metrics[bert-score]
Scoring failed for QA metric exact_match
Class: MissingDependencies
Message: evaluate package is not available. Please run pip install azureml-metrics[evaluate]
Failed to aggregate the scores for metric : bertscore with the following exception : ufunc 'add' did not contain a loop with signature matching types (dtype('<U19'), dtype('<U19')) -> None


[{'artifacts': {'gpt_coherence': ['5', '5', '1'],
                'gpt_fluency': ['5', '5', '1'],
                'gpt_groundedness': ['5', '5', '3'],
                'gpt_relevance': ['5', '5', '1'],
                'gpt_similarity': ['4', '4', '1']},
  'metrics': {'exact_match': nan, 'f1_score': 0.3763303533418476}}]


### Deploy Langchain QA Function to MIR

In [60]:
# download MLIndex files so they can be packaged with deployment code
client.mlindexes.download(name="product-info-cog-search-index", download_path="./qna_simple/mlindex", label="latest")

# set the deployment name to be used, the url needs to be globally unique so include project name
deployment_name = f"{client.project_name}-copilot" 

In [61]:
from azure.ai.generative.entities.deployment import Deployment
from azure.ai.generative.entities.models import LocalModel

deployment = Deployment(
    name=deployment_name,
    model=LocalModel(
        path="./qna_simple",
        conda_file="conda.yaml",
        loader_module="model_loader.py",
    ),
)

deployment = client.deployments.create_or_update(deployment)

/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Check: endpoint dantaylo-bugbashcli-copilot exists
Uploading mlflow_model (0.01 MBs): 100%|██████████| 10972/10972 [00:00<00:00, 20186.27it/s]




...............................................................................................................................

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


### Invoke Deployment

In [62]:
response = client.deployments.invoke(deployment_name, "./request_file_qna_simple.json")
print(response)

[{"question": "Which tent has the highest rainfly waterproof rating?", "answer": "The tent with the highest rainfly waterproof rating is the tent with item_number 8.", "context": "# Information about product item_number: 1\n\n# Information about product item_number: 1\n## Technical Specs\n**Best Use**: Camping  \n**Capacity**: 4-person  \n**Season Rating**: 3-season  \n**Setup**: Freestanding  \n**Material**: Polyester  \n**Waterproof**: Yes  \n**Floor Area**: 80 square feet  \n**Peak Height**: 6 feet  \n**Number of Doors**: 2  \n**Color**: Green  \n**Rainfly**: Included  \n**Rainfly Waterproof Rating**: 2000mm  \n**Tent Poles**: Aluminum  \n**Pole Diameter**: 9mm  \n**Ventilation**: Mesh panels and adjustable vents  \n**Interior Pockets**: Yes (4 pockets)  \n**Gear Loft**: Included  \n**Footprint**: Sold separately  \n**Guy Lines**: Reflective  \n**Stakes**: Aluminum  \n**Carry Bag**: Included  \n**Dimensions**: 10ft x 8ft x 6ft (length x width x peak height)  \n**Packed Size**: 24 in